In [3]:
import os


In [4]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification\\research'

In [5]:
os.chdir("../")

In [6]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Kidney_disease_classification'

In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Abhikkumar619/Kidney_disease_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Abhikkumar619"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b515b18fe70cac23bd1c8591a7c54e188845b00c"

In [8]:
import tensorflow as tf

In [9]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

In [11]:
@dataclass(frozen=True)
class EvaluationConfig: 
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [1]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:

        evaluation_config=EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Abhikkumar619/Kidney_disease_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        ) 
        return evaluation_config


In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from src.cnn_classifier import logger

In [20]:
class Evalutaion: 
    def __init__(self, config: EvaluationConfig): 
        self.config=config

    
    def train_valid_generator(self): 

        datagenerator_kwargs= dict(
            rescale=1./255,
            validation_split=0.30
            )
        logger.info(f"Train_valid generator : {datagenerator_kwargs}")
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        logger.info(f"Data flow kwargs : {dataflow_kwargs}")

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def save_score(self):
        scores={"loss": self.score[0], "accuracy": self.score[1]}
        logger.info(f"scores: {scores}")
        save_json(path=Path("scores.json"), data=scores)


    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self.train_valid_generator()
        self.score=model.evaluate(self.valid_generator)
        logger.info(f"Evaluation score {self.score}")
        self.save_score()
    

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
        if tracking_url_type_store !="file": 
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
        else: 
            mlflow.keras.log_model(self.model, "model")


In [30]:
try: 
    config=ConfigurationManager()
    model_evaluation_config=config.get_evaluation_config()
    evaluation=Evalutaion(config=model_evaluation_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e: 
    raise e

[2023-10-12 22:23:48,048 - INFO, - common - yaml file : config\config.yaml loaded sucessfully]
[2023-10-12 22:23:48,082 - INFO, - common - yaml file : params.yaml loaded sucessfully]
[2023-10-12 22:23:48,089 - INFO, - common - Created directories at : artifacts]
[2023-10-12 22:23:51,255 - INFO, - 2122144254 - Train_valid generator : {'rescale': 0.00392156862745098, 'validation_split': 0.3}]
[2023-10-12 22:23:51,257 - INFO, - 2122144254 - Data flow kwargs : {'target_size': [224, 224], 'batch_size': 16, 'interpolation': 'bilinear'}]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 189s 21s/step - loss: 22.6897 - accuracy: 0.4820
[2023-10-12 22:27:10,689 - INFO, - 2122144254 - Evaluation score [22.689699172973633, 0.48201438784599304]]
[2023-10-12 22:27:10,699 - INFO, - 2122144254 - scores: {'loss': 22.689699172973633, 'accuracy': 0.48201438784599304}]


2023/10/12 22:27:36 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-12 22:27:40,607 - WARNING, - save - Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\abish\AppData\Local\Temp\tmpnxdypqw2\model\data\model\assets
[2023-10-12 22:27:45,889 - INFO, - builder_impl - Assets written to: C:\Users\abish\AppData\Local\Temp\tmpnxdypqw2\model\data\model\assets]


c:\TheBritishCollege\DataScience\Kidney_disease_classification\kidney_vnv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.


MlflowException: API request to endpoint /api/2.0/mlflow/runs/get failed with error code 400 != 200. Response body: '"repo not associated with run"'

In [2]:
di={"key": "value1", "key2": "value2"}
save_json( data=di,path=Path("test12.json"))


In [15]:
di={"key": "value1", "key2": "value2"}
save_json( data=di,path=Path("test.json"))